In [1]:
import os
import sys
import glob

from tqdm import tqdm
import pandas as pd
import numpy as np
import ast
from scipy import stats

import plotly as py
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

sys.path.append(os.path.realpath(os.path.abspath('..')))
from iDrink import iDrinkUtilities
from iDrink.iDrinkUtilities import get_title_measure_name, get_unit, get_cad, get_setting_axis_name

In [2]:
drive = iDrinkUtilities.get_drivepath()

root_iDrink = os.path.join(drive, 'iDrink')
root_val = os.path.join(root_iDrink, "validation_root")
root_stat = os.path.join(root_val, '04_Statistics')
root_omc = os.path.join(root_val, '03_data', 'OMC_new', 'S15133')
root_data = os.path.join(root_val, "03_data")
root_logs = os.path.join(root_val, "05_logs")

csv_val_trials = os.path.join(root_logs, 'validation_trials.csv')
df_val_trials = pd.read_csv(csv_val_trials, sep=';')

csv_settings = os.path.join(root_logs, 'validation_settings.csv')
df_settings = pd.read_csv(csv_settings, sep=';')

csv_calib_error = os.path.join(root_logs, 'calib_errors.csv')
df_calib_error = pd.read_csv(csv_calib_error, sep=';')

csv_murphy = os.path.join(root_stat, '02_categorical', 'murphy_measures.csv')
df_murphy = pd.read_csv(csv_murphy, sep=';')

csv_failed_trials = os.path.join(root_stat, '04_failed_trials', 'failed_trials.csv')
df_failed_trials = pd.read_csv(csv_failed_trials, sep=';')

csv_ts_error = os.path.join(root_stat, '01_continuous', '01_results', 'omc_mmc_error.csv')
df_ts_error = pd.read_csv(csv_ts_error, sep=';')

csv_cad = os.path.join(root_stat, '02_categorical', 'clinically_acceptable_difference.csv')

df_cad = pd.read_csv(csv_cad, sep=',')

list_identifier = sorted(df_val_trials['identifier'].tolist())

ignore_id_p = ['P11', 'P19']
idx_s_singlecam_full = ['S017', 'S018', 'S019', 'S020', 'S021', 'S022', 'S023', 'S024', 'S025', 'S026']
idx_s_singlecam = ['S017', 'S018']
idx_s_multicam = ['S001', 'S002', 'S003', 'S004', 'S005', 'S006', 'S007', 'S008', 'S009', 'S010', 'S011', 'S012', 'S013', 'S014', 'S015', 'S016']
idx_s_multicam_reduced = ['S001', 'S002']
idx_s_reduced = idx_s_multicam_reduced + idx_s_singlecam
idx_s = idx_s_multicam + idx_s_singlecam

C:\Users\johan\AppData\Local\Temp\ipykernel_42352\303231503.py:26: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ts_error = pd.read_csv(csv_ts_error, sep=';')


In [3]:
df_ts_error = df_ts_error[(df_ts_error['id_s'].isin(idx_s_reduced)) & (~df_ts_error['id_p'].isin(ignore_id_p)) &
                          (df_ts_error['dynamic'] == 'fixed') & (df_ts_error['normalized'] == 'original') &  (df_ts_error['metric'] != 'trunk_ang')]

df_ts_error.insert(0, 'Setting', df_ts_error['id_s'].apply(lambda x: get_setting_axis_name(x)))
#df_ts_error.drop(columns=['id_s'], inplace=True)


df_ts_error['CAD'] = df_ts_error['metric'].apply(lambda x: get_cad(df_cad, x))
df_ts_error['metric'] = df_ts_error['metric'].apply(lambda x: get_title_measure_name(x, add_unit = True))
df_ts_error['RMSE < CAD'] = df_ts_error.apply(
    lambda row: 'Yes' if (row['rmse'] < row['CAD']) else 'No', axis=1)



In [4]:
df_ts_error_trial = df_ts_error[pd.notna(df_ts_error['id_t'])]
df_ts_error_patient = df_ts_error[(pd.isna(df_ts_error['id_t'])) & (pd.notna(df_ts_error['id_p']))]
df_ts_error_setting = df_ts_error[(pd.isna(df_ts_error['id_t'])) & (pd.isna(df_ts_error['id_p']))]

dir_out_trial = os.path.join(root_stat, '01_continuous', '02_plots', '06_rmse', '01_trial_rmse')
dir_out_patient = os.path.join(root_stat, '01_continuous', '02_plots', '06_rmse', '02_patient_rmse')
dir_out_setting = os.path.join(root_stat, '01_continuous', '02_plots', '06_rmse', '03_setting_rmse')

for d in [dir_out_trial, dir_out_patient, dir_out_setting]:
    os.makedirs(d, exist_ok=True)

In [5]:
df_ts_error.groupby(['Setting', 'metric']).median(numeric_only = True)

mean      median  \
Setting                  metric                                             
Metrabs, Cams: 1,2,3,4,5 Elbow Flexion [deg]        -3.461847   -3.228589   
                         Elbow Velocity [deg/s]      0.121226    0.023388   
                         Hand Velocity [mm/s]        4.057489    2.712659   
                         Shoulder Abduction [deg]   -1.282329   -1.371659   
                         Shoulder Flexion [deg]     -3.673221   -3.632949   
                         Trunk Displacement [mm]     1.772419    1.393937   
SimCC, Cams: 1,2,3,4,5   Elbow Flexion [deg]         2.792820    2.994323   
                         Elbow Velocity [deg/s]      0.101875    0.018582   
                         Hand Velocity [mm/s]       -1.647292   -0.639994   
                         Shoulder Abduction [deg]    6.358966    7.124213   
                         Shoulder Flexion [deg]     -5.859706   -6.301442   
                         Trunk Displacement [mm]     1.142454    0.766418   
Single, Cam: 1, filt     Elbow Flexion [deg]        -2.947925   -3.110017   
                         Elbow Velocity [deg/s]      0.121317    0.032030   
                         Hand Velocity [mm/s]       21.914644   19.537104   
                         Shoulder Abduction [deg]   -0.662418   -1.079577   
                         Shoulder Flexion [deg]     -4.065156   -3.985324   
                         Trunk Displacement [mm]     4.263552    3.633145   
Single, Cam: 1, unfilt   Elbow Flexion [deg]        -2.936641   -3.106472   
                         Elbow Velocity [deg/s]      0.160663    0.103548   
                         Hand Velocity [mm/s]      160.274522  152.672893   
                         Shoulder Abduction [deg]   -0.669922   -1.089133   
                         Shoulder Flexion [deg]     -4.100038   -3.996888   
                         Trunk Displacement [mm]     4.294249    3.658493   

                                                          std        rmse  \
Setting                  metric                                             
Metrabs, Cams: 1,2,3,4,5 Elbow Flexion [deg]         6.396001    7.970051   
                         Elbow Velocity [deg/s]      0.395245    0.411487   
                         Hand Velocity [mm/s]       55.389024   56.684139   
                         Shoulder Abduction [deg]    3.325136    4.085488   
                         Shoulder Flexion [deg]      5.431478    6.665371   
                         Trunk Displacement [mm]     5.732630    6.186305   
SimCC, Cams: 1,2,3,4,5   Elbow Flexion [deg]         6.259948    7.215898   
                         Elbow Velocity [deg/s]      0.351860    0.370485   
                         Hand Velocity [mm/s]       50.821128   51.206888   
                         Shoulder Abduction [deg]    6.878099    9.289592   
                         Shoulder Flexion [deg]      6.837763    9.103029   
                         Trunk Displacement [mm]     5.072222    5.613972   
Single, Cam: 1, filt     Elbow Flexion [deg]         6.426319    7.287284   
                         Elbow Velocity [deg/s]      0.390921    0.413259   
                         Hand Velocity [mm/s]       78.228769   81.978491   
                         Shoulder Abduction [deg]    5.203925    5.754212   
                         Shoulder Flexion [deg]      7.138788    8.844001   
                         Trunk Displacement [mm]    11.258011   12.155776   
Single, Cam: 1, unfilt   Elbow Flexion [deg]         6.442282    7.316155   
                         Elbow Velocity [deg/s]      0.387532    0.420033   
                         Hand Velocity [mm/s]      245.619891  299.367777   
                         Shoulder Abduction [deg]    5.197634    5.758135   
                         Shoulder Flexion [deg]      7.122169    8.823892   
                         Trunk Displacement [mm]    11.286346   12.230522   

                                                   

In [6]:
def prepare_df_ts_error_for_plot(df, mode='t'):
    df_new = df.copy()
    col_rename = {'id_p': 'Patient',
              'id_t': 'Trial',
              'condition': 'Condition',
              'metric': 'Metric',
              'rmse': 'RMSE',
              }

    if mode == 'p':
        group_list = ['Setting', 'id_s', 'Patient', 'Condition', 'Metric']
    elif mode == 's':
        group_list = ['Setting', 'id_s', 'Metric']
    else:
        group_list = ['Setting', 'id_s', 'Patient', 'Trial', 'Condition', 'Metric']

    df_new.rename(columns=col_rename, inplace=True)

    df_new = df_new.groupby(group_list, as_index = False).mean(numeric_only=True)
    df_new['RMSE < CAD'] = df_new.apply(
        lambda row: 'Yes' if (row['RMSE'] < row['CAD']) else 'No', axis=1)
    return df_new

df_to_plot_trial = prepare_df_ts_error_for_plot(df_ts_error_trial)
df_to_plot_patient = prepare_df_ts_error_for_plot(df_ts_error_patient, mode = 'p')
df_to_plot_setting = prepare_df_ts_error_for_plot(df_ts_error_setting, mode = 's')

## RMSE boxplot using trial_rmse

In [8]:
metrics = df_to_plot_trial['Metric'].unique()

for metric in metrics:
    df_to_plot_trial_metric = df_to_plot_trial[df_to_plot_trial['Metric'] == metric]
    fig = px.box(df_to_plot_trial_metric.sort_values(by='id_s'), x="Setting", y="RMSE",
                 template='plotly',
                 width=650, height=650)

    cad = df_to_plot_trial_metric['CAD'].unique()[0]
    fig.add_hline(y=cad, line_dash="dot", line_color='red', annotation_text="CAD", annotation_position="top right")

    title_name = get_title_measure_name(metric, add_unit = False)
    unit = get_unit(metric)

    fig.update_layout(title= dict(text=f'<b>{title_name} RMSE,<br>cad = {cad} {unit}<b>', font=dict(size=26)),
          xaxis_title=dict(text=f'<b>Settings<b>', font=dict(size=22)),
          yaxis_title=dict(text=f'<b>RMSE [{unit}]<b>', font=dict(size=22)),
              xaxis=dict(tickfont=dict(size=20)))

    os.makedirs(dir_out_trial, exist_ok=True)
    png_path = os.path.join(dir_out_trial, f'0306_RMSE_trial_box_{title_name}.png')
    fig.write_image(png_path, scale = 5)

    #fig.show()

## RMSE boxplot using patient_rmse

,Setting,id_s,Condition,Metric,mean,median,std,RMSE,rmse_std,CAD,RMSE < CAD
0,"Metrabs, Cams: 1,2,3,4,5",S002,affected,Elbow Flexion [deg],-3.871204,-3.510197,11.219871,12.369546,7.702405,4.48,No
1,"Metrabs, Cams: 1,2,3,4,5",S002,affected,Elbow Velocity [deg/s],0.160890,0.041248,0.527772,0.552846,0.422441,21.71,Yes
2,"Metrabs, Cams: 1,2,3,4,5",S002,affected,Hand Velocity [mm/s],56.022859,4.094845,271.347562,278.748447,227.912932,78.96,No
3,"Metrabs, Cams: 1,2,3,4,5",S002,affected,Shoulder Abduction [deg],-0.400327,-0.367538,4.674177,5.086991,2.876646,9.99,Yes
4,"Metrabs, Cams: 1,2,3,4,5",S002,affected,Shoulder Flexion [deg],-3.769974,-4.082235,7.853066,9.216502,5.556436,7.82,No
5,"Metrabs, Cams: 1,2,3,4,5",S002,affected,Trunk Displacement [mm],1.741753,1.462794,8.031723,8.599182,5.285753,31.89,Yes
6,"Metrabs, Cams: 1,2,3,4,5",S002,unaffected,Elbow Flexion [deg],-5.217339,-5.928237,11.660404,13.255576,7.802502,4.48,No
7,"Metrabs, Cams: 1,2,3,4,5",S002,unaffected,Elbow Velocity [deg/s],0.271663,0.043892,0.913004,0.954658,0.730622,21.71,Yes
8,"Metrabs, Cams: 1,2,3,4,5",S002,unaffected,Hand Velocity [mm/s],174.043624,8.477109,897.871498,919.089553,791.270396,78.96,No
9,"Metrabs, Cams: 1,2,3,4,5",S002,unaffected,Shoulder Abduction [deg],-1.638360,-1.619733,7.405452,8.628949,4.868363,9.99,Yes


## RMSE barplot using setting_rmse

In [24]:
metrics = df_to_plot_setting['Metric'].unique()

for metric in metrics:
    df_to_plot_setting_metric = df_to_plot_setting[df_to_plot_setting['Metric'] == metric]
    fig = px.bar(df_to_plot_setting_metric.sort_values(by='id_s'), x="Setting", y="RMSE",
                 template='plotly',
                 width=800, height=800)

    cad = df_to_plot_setting_metric['CAD'].unique()[0]
    fig.add_hline(y=cad, line_dash="dot", line_color='red', annotation_text="CAD", annotation_position="top right")

    title_name = get_title_measure_name(metric, add_unit = False)
    unit = get_unit(metric)

    fig.update_layout(title= dict(text=f'<b>{title_name} RMSE, cad = {cad} {unit}<b>', font=dict(size=24)),
          xaxis_title=dict(text=f'<b>Settings<b>', font=dict(size=20)),
          yaxis_title=dict(text=f'<b>RMSE [{unit}]<b>', font=dict(size=20)),
              xaxis=dict(tickfont=dict(size=20)))

    os.makedirs(dir_out_setting, exist_ok=True)
    png_path = os.path.join(dir_out_setting, f'0307_RMSE_setting_bar_{title_name}.png')
    fig.write_image(png_path, scale = 5

In [30]:
df_to_plot_setting

,Setting,Metric,RMSE,rmse_std,CAD,RMSE < CAD
0,"Metrabs, Cams: 1,2,3,4,5",Elbow Flexion [deg],12.812561,7.752453,4.48,No
1,"Metrabs, Cams: 1,2,3,4,5",Elbow Velocity [deg/s],0.753752,0.576531,21.71,Yes
2,"Metrabs, Cams: 1,2,3,4,5",Hand Velocity [mm/s],598.919000,509.591664,78.96,No
3,"Metrabs, Cams: 1,2,3,4,5",Shoulder Abduction [deg],6.857970,3.872504,9.99,Yes
4,"Metrabs, Cams: 1,2,3,4,5",Shoulder Flexion [deg],10.285400,6.024490,7.82,No
5,"Metrabs, Cams: 1,2,3,4,5",Trunk Displacement [mm],9.926016,6.064737,31.89,Yes
6,"SimCC, Cams: 1,2,3,4,5",Elbow Flexion [deg],13.800846,7.857575,4.48,No
7,"SimCC, Cams: 1,2,3,4,5",Elbow Velocity [deg/s],0.743271,0.591652,21.71,Yes
8,"SimCC, Cams: 1,2,3,4,5",Hand Velocity [mm/s],1174.945033,962.282915,78.96,No
9,"SimCC, Cams: 1,2,3,4,5",Shoulder Abduction [deg],13.496311,8.027766,9.99,No


In [29]:
df_to_plot_setting.drop(columns=['id_s', 'mean', 'median', 'std'], inplace=True)

KeyError: "['id_s', 'mean', 'median', 'std'] not found in axis"

In [32]:
print(df_to_plot_setting.round(2).style.to_latex(
    label='tab:res:cont:rmse_setting',
    caption=r'RMSE for 5-cam and single-cam settings. The full table can be found in section \ref{sec:app:plots_ts}'))

\begin{table}
\caption{RMSE for 5-cam and single-cam settings. The full table can be found in section \ref{sec:app:plots_ts}}
\label{tab:res:cont:rmse_setting}
\begin{tabular}{lllrrrl}
 & Setting & Metric & RMSE & rmse_std & CAD & RMSE < CAD \\
0 & Metrabs, Cams: 1,2,3,4,5 & Elbow Flexion [deg] & 12.810000 & 7.750000 & 4.480000 & No \\
1 & Metrabs, Cams: 1,2,3,4,5 & Elbow Velocity [deg/s] & 0.750000 & 0.580000 & 21.710000 & Yes \\
2 & Metrabs, Cams: 1,2,3,4,5 & Hand Velocity [mm/s] & 598.920000 & 509.590000 & 78.960000 & No \\
3 & Metrabs, Cams: 1,2,3,4,5 & Shoulder Abduction [deg] & 6.860000 & 3.870000 & 9.990000 & Yes \\
4 & Metrabs, Cams: 1,2,3,4,5 & Shoulder Flexion [deg] & 10.290000 & 6.020000 & 7.820000 & No \\
5 & Metrabs, Cams: 1,2,3,4,5 & Trunk Displacement [mm] & 9.930000 & 6.060000 & 31.890000 & Yes \\
6 & SimCC, Cams: 1,2,3,4,5 & Elbow Flexion [deg] & 13.800000 & 7.860000 & 4.480000 & No \\
7 & SimCC, Cams: 1,2,3,4,5 & Elbow Velocity [deg/s] & 0.740000 & 0.590000 & 21.71000